In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import holodeck.extensions

DUR = 3.0 * YR
CAD = 0.3 * YR
RESAMP = 2.0
TIME = 2.0 * GYR

fobs = holo.utils.nyquist_freqs(DUR, CAD)

fobs_orb = fobs / 2.0   # convert from GW-frequencies to orbital-frequencies (assuming circular)
realizer = holo.extensions.Realizer(fobs_orb, resample=RESAMP, lifetime=TIME, mamp=1e9*MSOL, mplaw=1.25)

In [ ]:
NREALS = 30
num_freq_bins = fobs.size - 1
# DOWN = None
DOWN = 2.0

gff = np.zeros((NREALS, num_freq_bins))
gwf = np.zeros_like(gff)
gwb = np.zeros_like(gff)
for rr in tqdm.trange(NREALS):
    names, samples = realizer(down_sample=DOWN)
    num_samp = samples[0].size
    weights = np.ones(num_samp)
    if DOWN is not None:
        weights *= DOWN
        
    hs, fo = holo.gravwaves._strains_from_samples(samples)
    gff[rr, :], gwf[rr, :], gwb[rr, :] = holo.gravwaves.gws_from_sampled_strains(fobs, fo, hs, weights)

In [ ]:
xx = fobs*YR
cc = kale.utils.midpoints(xx)
fig, ax = plot.figax()

med, *conf = np.percentile(gwb, [50, 25, 75], axis=0)
conf = [plot._get_hist_steps(xx, cc)[1] for cc in conf]

aa, bb = plot._get_hist_steps(xx, med)
ax.plot(aa, bb, alpha=0.65)

ax.fill_between(aa, *conf, alpha=0.2)

for yy in gwb:
    # aa, bb = plot._get_hist_steps(xx, yy)
    aa = cc; bb = yy
    ax.plot(aa, bb, 'k-', lw=0.5, alpha=0.5)

plt.show()